https://adventofcode.com/2021/day/24

In [1]:
from functools import lru_cache, partial
from itertools import product, combinations
from collections import defaultdict, Counter

In [2]:
with open('data/24.txt') as fh:
    data = fh.read()

In [3]:
class ALU:
    def __init__(self, prg='', vars=None):
        if vars is None:
            vars = {
                'w': 0,
                'x': 0,
                'y': 0,
                'z': 0,
            }
        self.v = vars
        self.prg = [y for y in (x.strip() for x in prg.split('\n')) if y]
        
    def ex(self, s, n=None):
        tokens = s.strip().split()
        fun, args = tokens[0], tokens[1:]
        if fun == 'inp':
            args.append(n)
        self.dispatch[fun](self, *args)

    def run(self, inp, wxyz=(0,0,0,0)):
        self.v['w'], self.v['x'], self.v['y'], self.v['z'] = wxyz
        it = iter(inp)
        for line in self.prg:
            if line.startswith('inp'):
                try:
                    nextit = next(it)
                except StopIteration:
                    return self.v['w'], self.v['x'], self.v['y'], self.v['z']
                self.ex(line, int(nextit))
            else:
                self.ex(line)
        return self.v['w'], self.v['x'], self.v['y'], self.v['z']
        

    def inp(self, a, n):
        self.v[a] = n

    def add(self, a, b):
        try:
            b = int(b)
        except ValueError:
            b = self.v[b]
        self.v[a] = self.v[a] + b

    def mul(self, a, b):
        try:
            b = int(b)
        except ValueError:
            b = self.v[b]
        self.v[a] = self.v[a] * b

    def div(self, a, b):
        try:
            b = int(b)
        except ValueError:
            b = self.v[b]
        self.v[a] = self.v[a] // b

    def mod(self, a, b):
        try:
            b = int(b)
        except ValueError:
            b = self.v[b]
        self.v[a] = self.v[a] % b

    def eql(self, a, b):
        try:
            b = int(b)
        except ValueError:
            b = self.v[b]
        self.v[a] = int(self.v[a] == b)

    dispatch = {
        'inp': inp,
        'add': add,
        'mul': mul,
        'div': div,
        'mod': mod,
        'eql': eql
}

In [4]:
data2 = data.replace('\ninp', '\nXX\ninp')
prgs = [x.strip() for x in data2.split('\nXX\n')]

In [5]:
len(prgs)

14

In [6]:
print(prgs[0])

inp w
mul x 0
add x z
mod x 26
div z 1
add x 15
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 15
mul y x
add z y


In [7]:
bigalu = ALU(data)

In [8]:
bigalu.run('99999999999999')

(9, 1, 10, 7587909014)

In [9]:
def params_from_prg(prg):
    prglines = [y for y in (x.strip() for x in prg.split('\n')) if y]
    return tuple(int(prglines[i].split()[-1]) for i in (4, 5, 15))

In [10]:
params = [params_from_prg(prg) for prg in prgs]

In [11]:
list(enumerate(params))

[(0, (1, 15, 15)),
 (1, (1, 12, 5)),
 (2, (1, 13, 6)),
 (3, (26, -14, 7)),
 (4, (1, 15, 9)),
 (5, (26, -7, 6)),
 (6, (1, 14, 14)),
 (7, (1, 15, 3)),
 (8, (1, 15, 1)),
 (9, (26, -7, 3)),
 (10, (26, -8, 4)),
 (11, (26, -7, 6)),
 (12, (26, -5, 7)),
 (13, (26, -10, 1))]

In [12]:
[(i, (d, a, b)) for (i, (d, a, b)) in enumerate(params) if a <= 9]

[(3, (26, -14, 7)),
 (5, (26, -7, 6)),
 (9, (26, -7, 3)),
 (10, (26, -8, 4)),
 (11, (26, -7, 6)),
 (12, (26, -5, 7)),
 (13, (26, -10, 1))]

In [13]:
[(i, (d, a, b)) for (i, (d, a, b)) in enumerate(params) if a > 9]

[(0, (1, 15, 15)),
 (1, (1, 12, 5)),
 (2, (1, 13, 6)),
 (4, (1, 15, 9)),
 (6, (1, 14, 14)),
 (7, (1, 15, 3)),
 (8, (1, 15, 1))]

In [14]:
max(b for (d, a, b) in params)

15

In [15]:
def fwd(w, z, d, a, b):
    r = z % 26
    z1 = z // d
    if r == w - a:
        return z1
    return z1 * 26 + w + b

In [16]:
def rev(w, z2, d, a, b):
    if d == 1:
        return [(z2 - w - b) // 26]
    
    z1 = z2 * 26
    L = []
    for r in range(26):
        z = z1 + r
        if fwd(w, z, d, a, b) == z2:
            L.append(z)
    
    return L

In [17]:
z = 0
for p in params:
    z = fwd(9, z, *p)
z

7587909014

In [18]:
rev(3, 0, *params[13])

[13]

In [19]:
funs = [partial(fwd, d=d, a=a, b=b) for (d, a, b) in params]

In [20]:
z = 0
zmaxs = [0]
for fun in funs:
    z = fun(9, z)
    zmaxs.append(z)
zmaxs = zmaxs[:-1]

rzmaxs = list(reversed(zmaxs))

In [21]:
rfuns = list(reversed(funs))

In [22]:
rparams = list(reversed(params))

In [23]:
# def r1(w, z2, d, a, b):
#     z = (z2 - w - b) / 26
#     return z

In [24]:
%%time
zs = [(0, [])]
for ((d, a, b), zmax) in zip(rparams[:5], rzmaxs):
    newzs = []
    for (z2, ws) in zs:
        for w in range(1, 10):
            for z in rev(w, z2, d, a, b):
                if z < zmax + 26:
                    newzs.append((z, ws + [w]))
    if not newzs:
        print('!')
        break
    zs = newzs
    print(len(zs))

9
81
729
6561
59049
CPU times: user 513 ms, sys: 10.9 ms, total: 524 ms
Wall time: 523 ms


9
81
729
6561
59049


In [25]:
min(zs), max(zs)

((5137842, [1, 1, 1, 1, 1]), (8939882, [9, 9, 9, 9, 9]))

In [26]:
zs.sort(reverse=True)

In [27]:
zs[:27]

[(8939882, [9, 9, 9, 9, 9]),
 (8939881, [9, 9, 9, 9, 8]),
 (8939880, [9, 9, 9, 9, 7]),
 (8939879, [9, 9, 9, 9, 6]),
 (8939878, [9, 9, 9, 9, 5]),
 (8939877, [9, 9, 9, 9, 4]),
 (8939876, [9, 9, 9, 9, 3]),
 (8939875, [9, 9, 9, 9, 2]),
 (8939874, [9, 9, 9, 9, 1]),
 (8939856, [9, 9, 9, 8, 9]),
 (8939855, [9, 9, 9, 8, 8]),
 (8939854, [9, 9, 9, 8, 7]),
 (8939853, [9, 9, 9, 8, 6]),
 (8939852, [9, 9, 9, 8, 5]),
 (8939851, [9, 9, 9, 8, 4]),
 (8939850, [9, 9, 9, 8, 3]),
 (8939849, [9, 9, 9, 8, 2]),
 (8939848, [9, 9, 9, 8, 1]),
 (8939830, [9, 9, 9, 7, 9]),
 (8939829, [9, 9, 9, 7, 8]),
 (8939828, [9, 9, 9, 7, 7]),
 (8939827, [9, 9, 9, 7, 6]),
 (8939826, [9, 9, 9, 7, 5]),
 (8939825, [9, 9, 9, 7, 4]),
 (8939824, [9, 9, 9, 7, 3]),
 (8939823, [9, 9, 9, 7, 2]),
 (8939822, [9, 9, 9, 7, 1])]

In [28]:
for z1, ws in zs:
    z = z1
    for w, p in zip(reversed(ws), params[-5:]):
        z = fwd(w, z, *p)
    if z!= 0:
        print(z1, ws)
        break
else:
    print('ok')

ok


In [29]:
len(zs)

59049

In [30]:
len(Counter(z for z, _ in zs))

59049

In [31]:
zend = {z: list(reversed(ws)) for z, ws in zs}

In [32]:
def fwd(w, z, d, a, b):
    r = z % 26
    z1 = z // d
    if r == w - a:
        return z1
    return z1 * 26 + w + b

In [75]:
%%time
zbegin = []
zprev = 0
for ws in product(range(1, 10), repeat=6):
    z = 0
    for w, p in zip(ws, params):
        z = fwd(w, z, *p)
    if z < zprev:
        zbegin.append((z, ws))
    zprev = z

CPU times: user 1.36 s, sys: 0 ns, total: 1.36 s
Wall time: 1.36 s


In [76]:
len(zbegin)

92502

In [77]:
zbegin[:10]

[(10980, (1, 1, 1, 1, 1, 3)),
 (285487, (1, 1, 1, 1, 2, 1)),
 (10980, (1, 1, 1, 1, 2, 4)),
 (285487, (1, 1, 1, 1, 3, 1)),
 (10980, (1, 1, 1, 1, 3, 5)),
 (285487, (1, 1, 1, 1, 4, 1)),
 (10980, (1, 1, 1, 1, 4, 6)),
 (285487, (1, 1, 1, 1, 5, 1)),
 (10980, (1, 1, 1, 1, 5, 7)),
 (285487, (1, 1, 1, 1, 6, 1))]

In [78]:
len(Counter(z for (z, _) in zbegin))

1620

In [82]:
zbeginmaxs = {}
for z, ws in zbegin:
    zbeginmaxs[z] = max(ws, zbeginmaxs.get(z, (0,0,0,0,0,0)))

In [83]:
zmids = []
for b in zbeginmaxs:
    for ws in product(range(1, 10), repeat=3):
        z = b
        for w, p in zip(ws, params[6:]):
            z = fwd(w, z, *p)
        if z in zend:
            zmids.append(((b, z), ws))
            
            
        
        

In [84]:
len(zmids)

864

In [85]:
zmids[:10]

[((422, 7427454), (1, 6, 7)),
 ((422, 7427455), (1, 6, 8)),
 ((422, 7427456), (1, 6, 9)),
 ((422, 7427480), (1, 7, 7)),
 ((422, 7427481), (1, 7, 8)),
 ((422, 7427482), (1, 7, 9)),
 ((422, 7427506), (1, 8, 7)),
 ((422, 7427507), (1, 8, 8)),
 ((422, 7427508), (1, 8, 9)),
 ((422, 7427532), (1, 9, 7))]

In [86]:
len(Counter(x for (x, _) in zmids))

864

In [87]:
zmidset = set(a for ((a, _), _) in zmids)

In [88]:
len(zmidset)

36

In [91]:
maxb, maxws = (0, (0,0,0,0,0,0))
for b, ws in zbegin:
    if b in zmidset and ws > maxws:
        maxb, maxws = b, ws

In [92]:
maxb, maxws

(508, (4, 9, 9, 1, 7, 9))

In [93]:
goodbs = [((b, e), ws) for ((b, e), ws) in zmids if b == 508]

In [94]:
type(zend)

dict

In [46]:
list(zend.items())[:3]

[(8939882, [9, 9, 9, 9, 9]),
 (8939881, [8, 9, 9, 9, 9]),
 (8939880, [7, 9, 9, 9, 9])]

In [95]:
for ((b, e), ws) in goodbs:
    if e in zend:
        print(b, e, ws, zend[e])

508 8938990 (1, 6, 7) [1, 1, 8, 9, 9]
508 8938991 (1, 6, 8) [2, 1, 8, 9, 9]
508 8938992 (1, 6, 9) [3, 1, 8, 9, 9]
508 8939016 (1, 7, 7) [1, 2, 8, 9, 9]
508 8939017 (1, 7, 8) [2, 2, 8, 9, 9]
508 8939018 (1, 7, 9) [3, 2, 8, 9, 9]
508 8939042 (1, 8, 7) [1, 3, 8, 9, 9]
508 8939043 (1, 8, 8) [2, 3, 8, 9, 9]
508 8939044 (1, 8, 9) [3, 3, 8, 9, 9]
508 8939068 (1, 9, 7) [1, 4, 8, 9, 9]
508 8939069 (1, 9, 8) [2, 4, 8, 9, 9]
508 8939070 (1, 9, 9) [3, 4, 8, 9, 9]
508 8939666 (2, 6, 7) [1, 1, 9, 9, 9]
508 8939667 (2, 6, 8) [2, 1, 9, 9, 9]
508 8939668 (2, 6, 9) [3, 1, 9, 9, 9]
508 8939692 (2, 7, 7) [1, 2, 9, 9, 9]
508 8939693 (2, 7, 8) [2, 2, 9, 9, 9]
508 8939694 (2, 7, 9) [3, 2, 9, 9, 9]
508 8939718 (2, 8, 7) [1, 3, 9, 9, 9]
508 8939719 (2, 8, 8) [2, 3, 9, 9, 9]
508 8939720 (2, 8, 9) [3, 3, 9, 9, 9]
508 8939744 (2, 9, 7) [1, 4, 9, 9, 9]
508 8939745 (2, 9, 8) [2, 4, 9, 9, 9]
508 8939746 (2, 9, 9) [3, 4, 9, 9, 9]


In [96]:
attempt = [4, 9, 9, 1, 7, 9, 2, 9, 9, 3, 4, 9, 9, 9]

In [97]:
z = 0
for w, p in zip(attempt, params):
    z = fwd(w, z, *p)
z

0

In [98]:
part1 = ''.join(str(x) for x in attempt)
print(part1)

49917929934999


## Part 2

In [103]:
zbeginmins = {}
for z, ws in zbegin:
    zbeginmins[z] = min(ws, zbeginmins.get(z, (9,9,9,9,9,9)))

In [104]:
minb, minws = maxb, maxws
for b, ws in zbeginmins.items():
    if b in zmidset and ws < minws:
        minb, minws = b, ws

In [105]:
minb, minws

(422, (1, 1, 9, 1, 1, 3))

In [106]:
minbs = [((b, e), ws) for ((b, e), ws) in zmids if b == minb]

In [107]:
for ((b, e), ws) in minbs:
    if e in zend:
        print(b, e, ws, zend[e])

422 7427454 (1, 6, 7) [1, 1, 8, 1, 6]
422 7427455 (1, 6, 8) [2, 1, 8, 1, 6]
422 7427456 (1, 6, 9) [3, 1, 8, 1, 6]
422 7427480 (1, 7, 7) [1, 2, 8, 1, 6]
422 7427481 (1, 7, 8) [2, 2, 8, 1, 6]
422 7427482 (1, 7, 9) [3, 2, 8, 1, 6]
422 7427506 (1, 8, 7) [1, 3, 8, 1, 6]
422 7427507 (1, 8, 8) [2, 3, 8, 1, 6]
422 7427508 (1, 8, 9) [3, 3, 8, 1, 6]
422 7427532 (1, 9, 7) [1, 4, 8, 1, 6]
422 7427533 (1, 9, 8) [2, 4, 8, 1, 6]
422 7427534 (1, 9, 9) [3, 4, 8, 1, 6]
422 7428130 (2, 6, 7) [1, 1, 9, 1, 6]
422 7428131 (2, 6, 8) [2, 1, 9, 1, 6]
422 7428132 (2, 6, 9) [3, 1, 9, 1, 6]
422 7428156 (2, 7, 7) [1, 2, 9, 1, 6]
422 7428157 (2, 7, 8) [2, 2, 9, 1, 6]
422 7428158 (2, 7, 9) [3, 2, 9, 1, 6]
422 7428182 (2, 8, 7) [1, 3, 9, 1, 6]
422 7428183 (2, 8, 8) [2, 3, 9, 1, 6]
422 7428184 (2, 8, 9) [3, 3, 9, 1, 6]
422 7428208 (2, 9, 7) [1, 4, 9, 1, 6]
422 7428209 (2, 9, 8) [2, 4, 9, 1, 6]
422 7428210 (2, 9, 9) [3, 4, 9, 1, 6]


In [55]:
part2 = '11911316711816'